### Arches QuerySets tutorial

This tutorial shows the approach `arches-querysets` uses to adapt the Arches data model to the Django ORM (and thus the rest of the Django ecosystem).

#### Test data
In this tutorial, we'll use a sample resource model called "Datatype Lookups". Set up the test data. You should replace the settings module below with the one you wish to use for this tutorial. Run `manage.py setup_db` first if necessary. (This is not included in the notebook out of caution.)

In [1]:
import os

import django
from django.core.management import call_command

# Simulate manage.py
os.environ["DJANGO_SETTINGS_MODULE"] = "arches_querysets.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

call_command("delete_test_data")
call_command("add_test_data")

Deleting test data...
Finished!
Creating test data...
Finished!


In [3]:
from arches_querysets.models import *

resource = ResourceTileTree.get_tiles(graph_slug="datatype_lookups").exclude(url_alias=None).get()
print(resource.aliased_data)
print(resource.aliased_data.datatypes_1.aliased_data)

AliasedData(datatypes_1=<TileTree: datatypes_1 (15459169-79b0-47a8-9cc3-cddbfb849298)>, datatypes_n=[<TileTree: datatypes_n (dad5c8eb-4f4f-4f69-887d-1e959589b255)>])
AliasedData(concept_alias=<Value: Value object (d8c60bf4-e786-11e6-905a-b756ec83dad5)>, geojson_feature_collection_alias=None, concept_list_alias=[<Value: Value object (d8c60bf4-e786-11e6-905a-b756ec83dad5)>], number_alias=42, resource_instance_alias=<ResourceInstance: Datatype Lookups: Resource referencing 42 (d19bbde5-a8ac-40c3-af46-96e375e322ed)>, resource_instance_list_alias=[<ResourceInstance: Datatype Lookups: Resource referencing 42 (d19bbde5-a8ac-40c3-af46-96e375e322ed)>], file_list_alias=[{'url': 'http://www.archesproject.org/blog/static/42.png', 'name': '42_accessibility_improvements.png', 'size': 2042, 'type': 'image/png', 'index': 0, 'title': {'en': {'value': '42 Accessibility Improvements', 'direction': 'ltr'}}, 'width': 2042, 'height': 2042, 'status': 'added', 'altText': {'en': {'value': 'Illustration of rece

In [5]:
resource = ResourceTileTree.get_tiles(
    graph_slug="datatype_lookups", as_representation=True
).exclude(url_alias=None).get()

from pprint import pprint
pprint(resource.aliased_data)
pprint(resource.aliased_data.datatypes_1.aliased_data)

AliasedData(datatypes_1=<TileTree: datatypes_1 (15459169-79b0-47a8-9cc3-cddbfb849298)>,
            datatypes_n=[<TileTree: datatypes_n (dad5c8eb-4f4f-4f69-887d-1e959589b255)>])
AliasedData(concept_alias={'details': [{'concept_id': '00000000-0000-0000-0000-000000000001',
                                        'language_id': 'en',
                                        'value': 'Arches',
                                        'valueid': 'd8c60bf4-e786-11e6-905a-b756ec83dad5',
                                        'valuetype_id': 'prefLabel'}],
                           'display_value': 'Arches',
                           'node_value': 'd8c60bf4-e786-11e6-905a-b756ec83dad5'},
            geojson_feature_collection_alias={'details': [],
                                              'display_value': '(Empty)',
                                              'node_value': None},
            concept_list_alias={'details': [{'concept_id': '00000000-0000-0000-0000-000000000001',
         

In [6]:
test_resources = ResourceTileTree.get_tiles("datatype_lookups", as_representation=True)
# This "shallow" lookup works whether or not the node is nested (todo: add test case)
resource = test_resources.filter(string_alias__any_lang_contains="forty").first()
resource.aliased_data.datatypes_1.aliased_data.string_alias

{'node_value': {'en': {'value': 'forty-two', 'direction': 'ltr'}},
 'display_value': 'forty-two',
 'details': []}

In [7]:
# You can save back with or without the display value / node_value wrapper
resource.aliased_data.datatypes_1.aliased_data.string_alias = 'forty-three'
resource.save()
# But without a request, we assume anonymous user, and it ends up in provisional edits
resource.aliased_data.datatypes_1.provisionaledits

{'2': {'value': {'00091961-24b0-43c2-9169-b7b77ff898bd': None,
   '039bdb19-7a21-463a-9308-9101b072f07a': ['d8c60bf4-e786-11e6-905a-b756ec83dad5'],
   '35d7f2a3-471a-441d-8524-034f3096343d': [{'resourceId': 'd19bbde5-a8ac-40c3-af46-96e375e322ed',
     'ontologyProperty': '',
     'resourceXresourceId': '0717484e-9f65-4a2d-93e3-edbbbfaca0ed',
     'inverseOntologyProperty': ''}],
   '368c027d-628c-448f-8e60-e83780ea74fe': [{'resourceId': 'd19bbde5-a8ac-40c3-af46-96e375e322ed',
     'ontologyProperty': '',
     'resourceXresourceId': 'e9f3a337-600a-41a0-bf86-fee797c8327b',
     'inverseOntologyProperty': ''}],
   '4130e176-64e3-4b10-b0fb-5a2fad7bb19a': '15459169-79b0-47a8-9cc3-cddbfb849298',
   '5785a336-b6d0-4046-85df-31d329779502': {'url': 'http://www.42.com/',
    'url_label': '42.com'},
   '5a1e1c7d-18df-45e3-b3c9-551f18b0c751': 'forty-two',
   '7a98f64a-6db7-4069-9790-97bc0bf56d2a': [{'url': '/files/588263f6-4aec-4581-a0c2-f05c2b6c8f0a',
     'name': '42_accessibility_improvements.p

In [8]:
resource.aliased_data.datatypes_1.aliased_data.string_alias = 'forty-three'
resource.save(force_admin=True)
resource.aliased_data.datatypes_1.aliased_data.string_alias

{'node_value': {'en': {'value': 'forty-three', 'direction': 'ltr'}},
 'display_value': 'forty-three',
 'details': []}

In [9]:
# Custom lookups
resources = ResourceTileTree.get_tiles(
    "datatype_lookups", as_representation=True
).filter(string_alias__any_lang_contains="forty")
for match in resources:
    print(match)

Datatype Lookups: Resource referencing 42 (d19bbde5-a8ac-40c3-af46-96e375e322ed)


In [10]:
resources = ResourceTileTree.get_tiles(
    "datatype_lookups", as_representation=True
).filter(resource_instance_list_alias__contains=str(resource.pk))
for match in resources:
    print(match)

Datatype Lookups: Resource referencing 42 (d19bbde5-a8ac-40c3-af46-96e375e322ed)
